In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import timedelta, datetime


In [2]:
# set option to display precision to 10 decimal places
pd.set_option('display.precision', 10)

In [3]:
df_crypto = pd.read_csv('listoneprice22(1).csv')
df_index = pd.read_csv('crix_index(1).csv')

In [4]:
print(df_crypto.head())
print(df_index.head())

   code      Date   close
0  ABBC  10/12/18  0.3714
1  ABBC  10/13/18  0.3760
2  ABBC  10/14/18  0.3631
3  ABBC  10/15/18  0.3616
4  ABBC  10/16/18  0.5856
      date  price_index
0  7/31/14  1000.000000
1   8/1/14  1018.202717
2   8/2/14  1008.772389
3   8/3/14  1004.416500
4   8/4/14  1004.984138


In [5]:

# convert date to datetime
df_crypto['Date'] = pd.to_datetime(df_crypto['Date'])
df_index['date'] = pd.to_datetime(df_index['date'])



/var/folders/cc/b5_0zmwd6bs8ktgj2_7zlnnm0000gn/T/ipykernel_18561/3028583592.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_crypto['Date'] = pd.to_datetime(df_crypto['Date'])
/var/folders/cc/b5_0zmwd6bs8ktgj2_7zlnnm0000gn/T/ipykernel_18561/3028583592.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_index['date'] = pd.to_datetime(df_index['date'])


In [6]:
# Rename  coluns date to match
df_index.rename(columns={'date': 'Date'}, inplace=True)

In [7]:
# merge the dataframces on the 'Date' column
df = pd.merge(df_crypto, df_index, on='Date', how='inner')

In [8]:
df.to_csv('merged.csv', index=False)
df

,code,Date,close,price_index
0,ABBC,2018-10-12,0.3714,15050.19052
1,ABBC,2018-10-13,0.3760,15355.43717
2,ABBC,2018-10-14,0.3631,15470.59004
3,ABBC,2018-10-15,0.3616,15342.49212
4,ABBC,2018-10-16,0.5856,16276.02719
...,...,...,...,...
497494,ZRX,2021-07-09,0.8406,93266.37697
497495,ZRX,2021-07-10,0.7891,95906.13394
497496,ZRX,2021-07-11,0.7947,95095.76486
497497,ZRX,2021-07-12,0.7543,96639.42235


In [9]:
# rename columns to match code
df.rename(columns={'Date': 'Date', 'close': 'crypto_prices', 'price_index': 'market_prices'}, inplace=True)
df.to_csv('merged.csv', index=False)
df

,code,Date,crypto_prices,market_prices
0,ABBC,2018-10-12,0.3714,15050.19052
1,ABBC,2018-10-13,0.3760,15355.43717
2,ABBC,2018-10-14,0.3631,15470.59004
3,ABBC,2018-10-15,0.3616,15342.49212
4,ABBC,2018-10-16,0.5856,16276.02719
...,...,...,...,...
497494,ZRX,2021-07-09,0.8406,93266.37697
497495,ZRX,2021-07-10,0.7891,95906.13394
497496,ZRX,2021-07-11,0.7947,95095.76486
497497,ZRX,2021-07-12,0.7543,96639.42235


In [10]:
"""
Description: This function calculates the compounded weekly returns for a given time series of daily prices.
Parameters:
    daily_prices: pd.Series - A time series of daily prices

Note:
    This function will take in the daily prices and calculate the daily returns.
    It will then resample the daily returns to weekly, with weeks ending on Wednesday.
    The weekly returns are then compounded to get the compounded weekly returns.
"""
def calculate_compounded_returns(daily_prices):
    # Calculate daily returns
    daily_returns = daily_prices.pct_change().add(1)

    # Resample to weekly, with weeks ending on Wednesday
    # Calculate the product of returns for each week to get compounded return
    weekly_compounded_returns = daily_returns.resample('W-WED').prod().sub(1)

    return weekly_compounded_returns


In [11]:
# Putting this into a function
# adding weekly returns
def add_weekly_returns(df):
    

    # Convert 'Date' to datetime format and set as index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    # Lists to store the results
    results = []
    
    # Loop through each crypto in the dataframe
    for code, group in df.groupby('code'):
        # Calculate compounded returns for each group
        group['crypto_weekly_returns'] = calculate_compounded_returns(group['crypto_prices'])
        group['market_weekly_returns'] = calculate_compounded_returns(group['market_prices'])
        
        results.append(group)
        
    # Concatenate the results and reset the index
    df = pd.concat(results)
    
    return df


In [12]:
df

,code,Date,crypto_prices,market_prices
0,ABBC,2018-10-12,0.3714,15050.19052
1,ABBC,2018-10-13,0.3760,15355.43717
2,ABBC,2018-10-14,0.3631,15470.59004
3,ABBC,2018-10-15,0.3616,15342.49212
4,ABBC,2018-10-16,0.5856,16276.02719
...,...,...,...,...
497494,ZRX,2021-07-09,0.8406,93266.37697
497495,ZRX,2021-07-10,0.7891,95906.13394
497496,ZRX,2021-07-11,0.7947,95095.76486
497497,ZRX,2021-07-12,0.7543,96639.42235


In [13]:
# Add weekly returns
df = add_weekly_returns(df)

In [14]:
df.to_csv('test.csv', index=False)
df

,code,crypto_prices,market_prices,crypto_weekly_returns,market_weekly_returns
Date,,,,,
2018-10-12,ABBC,0.3714,15050.19052,NaN,NaN
2018-10-13,ABBC,0.3760,15355.43717,NaN,NaN
2018-10-14,ABBC,0.3631,15470.59004,NaN,NaN
2018-10-15,ABBC,0.3616,15342.49212,NaN,NaN
2018-10-16,ABBC,0.5856,16276.02719,NaN,NaN
...,...,...,...,...,...
2021-07-09,ZRX,0.8406,93266.37697,NaN,NaN
2021-07-10,ZRX,0.7891,95906.13394,NaN,NaN
2021-07-11,ZRX,0.7947,95095.76486,NaN,NaN


In [15]:
# Drop na values
df.dropna(inplace=True)
df.to_csv('test.csv', index=False)
df

,code,crypto_prices,market_prices,crypto_weekly_returns,market_weekly_returns
Date,,,,,
2018-10-17,ABBC,0.9843,16328.89946,1.6502423263,0.0849629736
2018-10-24,ABBC,0.5290,15882.57172,-0.4625622270,-0.0273336082
2018-10-31,ABBC,0.3924,15461.70751,-0.2582230624,-0.0264984927
2018-11-07,ABBC,0.3708,16751.11120,-0.0550458716,0.0833933567
2018-11-14,ABBC,0.3276,16074.78232,-0.1165048544,-0.0403751651
...,...,...,...,...,...
2021-06-09,ZRX,0.9647,97209.27754,-0.1611304348,-0.0786687895
2021-06-16,ZRX,0.8670,113754.54710,-0.1012750078,0.1702025771
2021-06-23,ZRX,0.6391,90074.02464,-0.2628604383,-0.2081720957


In [16]:
def calculate_lagged_returns(df):
    results = []

    # Group by 'code' and calculate lagged returns for each group
    for code, group in df.groupby('code'):
        # Calculate Lagged Returns
        for n in range(1, 5):
            group[f'market_returns_lag_{n}'] = group['market_weekly_returns'].shift(n)

        # Remove rows with NaN values which result from lagging
        group.dropna(inplace=True)

        # Append the group to the results list
        results.append(group)

    # Concatenate all the groups back into a single DataFrame
    df = pd.concat(results)

    # Save to CSV if needed
    df.to_csv('data_with_lags.csv')

    return df


In [17]:
# Calculate lagged returns
df = calculate_lagged_returns(df)

In [18]:
df

,code,crypto_prices,market_prices,crypto_weekly_returns,market_weekly_returns,market_returns_lag_1,market_returns_lag_2,market_returns_lag_3,market_returns_lag_4
Date,,,,,,,,,
2018-11-14,ABBC,0.32760,16074.782320,-0.1165048544,-0.0403751651,0.0833933567,-0.0264984927,-0.0273336082,0.0849629736
2018-11-21,ABBC,0.13720,11028.630650,-0.5811965812,-0.3139172630,-0.0403751651,0.0833933567,-0.0264984927,-0.0273336082
2018-11-28,ABBC,0.05668,9473.017754,-0.5868804665,-0.1410522254,-0.3139172630,-0.0403751651,0.0833933567,-0.0264984927
2018-12-05,ABBC,0.03304,9513.722081,-0.4170783345,0.0042968701,-0.1410522254,-0.3139172630,-0.0403751651,0.0833933567
2018-12-12,ABBC,0.05869,7985.814608,0.7763317191,-0.1606003896,0.0042968701,-0.1410522254,-0.3139172630,-0.0403751651
...,...,...,...,...,...,...,...,...,...
2021-06-09,ZRX,0.96470,97209.277540,-0.1611304348,-0.0786687895,-0.0311382844,-0.1187782386,-0.2462079971,0.0826278905
2021-06-16,ZRX,0.86700,113754.547100,-0.1012750078,0.1702025771,-0.0786687895,-0.0311382844,-0.1187782386,-0.2462079971
2021-06-23,ZRX,0.63910,90074.024640,-0.2628604383,-0.2081720957,0.1702025771,-0.0786687895,-0.0311382844,-0.1187782386


In [19]:
df.reset_index(drop=False, inplace=True)
df

,Date,code,crypto_prices,market_prices,crypto_weekly_returns,market_weekly_returns,market_returns_lag_1,market_returns_lag_2,market_returns_lag_3,market_returns_lag_4
0,2018-11-14,ABBC,0.32760,16074.782320,-0.1165048544,-0.0403751651,0.0833933567,-0.0264984927,-0.0273336082,0.0849629736
1,2018-11-21,ABBC,0.13720,11028.630650,-0.5811965812,-0.3139172630,-0.0403751651,0.0833933567,-0.0264984927,-0.0273336082
2,2018-11-28,ABBC,0.05668,9473.017754,-0.5868804665,-0.1410522254,-0.3139172630,-0.0403751651,0.0833933567,-0.0264984927
3,2018-12-05,ABBC,0.03304,9513.722081,-0.4170783345,0.0042968701,-0.1410522254,-0.3139172630,-0.0403751651,0.0833933567
4,2018-12-12,ABBC,0.05869,7985.814608,0.7763317191,-0.1606003896,0.0042968701,-0.1410522254,-0.3139172630,-0.0403751651
...,...,...,...,...,...,...,...,...,...,...
69179,2021-06-09,ZRX,0.96470,97209.277540,-0.1611304348,-0.0786687895,-0.0311382844,-0.1187782386,-0.2462079971,0.0826278905
69180,2021-06-16,ZRX,0.86700,113754.547100,-0.1012750078,0.1702025771,-0.0786687895,-0.0311382844,-0.1187782386,-0.2462079971
69181,2021-06-23,ZRX,0.63910,90074.024640,-0.2628604383,-0.2081720957,0.1702025771,-0.0786687895,-0.0311382844,-0.1187782386
69182,2021-06-30,ZRX,0.74010,100747.632100,0.1580347363,0.1184981742,-0.2081720957,0.1702025771,-0.0786687895,-0.0311382844


In [20]:

# df = pd.DataFrame({'Date': dates, 'crypto_ID': crypto_ids, 'crypto_prices': crypto_prices, 
#                    'market_prices': market_prices, 'crypto_weekly_returns': crypto_returns, 
#                    'market_returns_lag_1': market_lag1, 'market_returns_lag_2': market_lag2, 
#                    'market_returns_lag_3': market_lag3, 'market_returns_lag_4': market_lag4})

def calculate_annual_delay_measures(df):
    results = []

    # Assuming df is your DataFrame and 'Date' is in a format that pandas can recognize as a date
    df['Date'] = pd.to_datetime(df['Date'])

    # Function to adjust the year
    def adjust_year(row):
        if row.month < 7:  # If month is before July
            return row.year - 1  # Consider it as part of the previous year
        else:
            return row.year

    # First group by cryptocurrency
    for code, crypto_group in df.groupby('code'):
        # Convert 'Date' to datetime and apply the year adjustment within each crypto group
        crypto_group['Date'] = pd.to_datetime(crypto_group['Date'])
        crypto_group['Year'] = crypto_group['Date'].apply(adjust_year)

        # Now group by year within each crypto group
        for year, year_group in crypto_group.groupby('Year'):
            
            # Skip adjusted years with less than 52 weeks
            if len(year_group) < 52:
                continue
            
            
            # Prepare regression variables
            X = sm.add_constant(year_group[['market_weekly_returns', 'market_returns_lag_1', 'market_returns_lag_2', 'market_returns_lag_3', 'market_returns_lag_4']])
            y = year_group['crypto_weekly_returns']

            # Full regression model
            full_model = sm.OLS(y, X).fit()
            r_squared_full = full_model.rsquared

            # Restricted regression model (only contemporaneous market return)
            X_restricted = sm.add_constant(year_group[['market_weekly_returns']])
            restricted_model = sm.OLS(y, X_restricted).fit()
            r_squared_restricted = restricted_model.rsquared

            # Calculate D1
            D1 = 1 - (r_squared_restricted / r_squared_full)

            # Append results for each year and crypto
            results.append({'code': code, 'Year': year, 'D1': D1})
            
    return pd.DataFrame(results)

# Usage example
annual_delay_measures = calculate_annual_delay_measures(df)
print(annual_delay_measures)

/Users/ranystephan/LocalDocs/Projects/pricedelay/testvenv/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/Users/ranystephan/LocalDocs/Projects/pricedelay/testvenv/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/Users/ranystephan/LocalDocs/Projects/pricedelay/testvenv/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1733: RuntimeWarning: invalid value encountered in subtract
  return np.sum(weights * (model.endog - mean)**2)
/Users/ranystephan/LocalDocs/Projects/pricedelay/testvenv/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1733: RuntimeWarning: invalid value encountered in subtract
  return np.sum(weights * (model.endog - mean)**2)
/Users/ranystephan/LocalDocs/Projects/pricedelay/testvenv/lib/py

      code  Year            D1
0     ABBC  2019  0.7931434554
1     ABBC  2020  0.7350940201
2      ACT  2018  0.0687544833
3      ACT  2019  0.0439462407
4      ACT  2020  0.6612492258
...    ...   ...           ...
1060   ZIL  2020  0.3156204044
1061   ZNN  2020  0.3547699308
1062   ZRX  2018  0.0986147677
1063   ZRX  2019  0.3426985458
1064   ZRX  2020  0.1497979811

[1065 rows x 3 columns]


In [21]:
# export to csv

df.to_csv('final_df_after_computation.csv', index=False)

In [22]:
annual_delay_measures.to_csv('annual_delay_measures.csv', index=False)
annual_delay_measures

,code,Year,D1
0,ABBC,2019,0.7931434554
1,ABBC,2020,0.7350940201
2,ACT,2018,0.0687544833
3,ACT,2019,0.0439462407
4,ACT,2020,0.6612492258
...,...,...,...
1060,ZIL,2020,0.3156204044
1061,ZNN,2020,0.3547699308
1062,ZRX,2018,0.0986147677
1063,ZRX,2019,0.3426985458
